In [2]:
import pandas as pd
#headers:timestamp,t,day,dt_h,hour_of_day,month,season,dayofyear,t_out_c,price_eur_per_kwh,solar_gen_kw_per_kwp,in_work_hours
path_1 = r"C:\_projekty_repo\noc\thermal_game\data\prices_15min.csv"
path2 = r"C:\_projekty_repo\noc\thermal_game\data\weather_15min.csv"

df_1 = pd.read_csv(path_1,index_col=0,)
df_2 = pd.read_csv(path2, index_col=0,)


In [ ]:
import pandas as pd
import numpy as np

# --- inputs you already have ---
# path_1 = r"C:\_projekty_repo\noc\thermal_game\data\prices_15min.csv"
# path2  = r"C:\_projekty_repo\noc\thermal_game\data\weather_15min.csv"
# df_1 = pd.read_csv(path_1, index_col=0)
# df_2 = pd.read_csv(path2,  index_col=0)

# ---------- 0) Ensure datetime index, tidy ----------
for _df in (df_1, df_2):
    if not isinstance(_df.index, pd.DatetimeIndex):
        _df.index = pd.to_datetime(_df.index, errors="coerce", utc=False)
    _df.sort_index(inplace=True)
    _df.rename(columns=str.strip, inplace=True)

# ---------- 1) Select year 2023 ----------
p23 = df_1[df_1.index.year == 2023].copy()
w23 = df_2[df_2.index.year == 2023].copy()

if p23.empty or w23.empty:
    raise ValueError("No 2023 data found in one or both sources (prices/weather).")

# ---------- 2) Change year from 2023 -> 2025 (keep month/day/time) ----------
# (2023 and 2025 are both non-leap years, so this is safe.)
p25 = p23.copy()
w25 = w23.copy()
p25.index = p25.index.map(lambda ts: ts.replace(year=2025))
w25.index = w25.index.map(lambda ts: ts.replace(year=2025))

# Optional: if you might collide with existing 2025 rows, keep the 2023->2025 relabeled data
# p25 = p25[~p25.index.isin(df_1[df_1.index.year == 2025].index)]
# w25 = w25[~w25.index.isin(df_2[df_2.index.year == 2025].index)]

# ---------- 3) Join on the relabeled 2025 timestamps ----------
df = p25.join(w25, how="inner")
df.index.name = "timestamp"
if df.empty:
    raise ValueError("After relabeling to 2025, there are no overlapping timestamps between price and weather.")

# ---------- 4) Derive requested features ----------
idx = df.index

# dt_h: detect step, default 0.25 (15 min)
dt_series = idx.to_series().diff().dt.total_seconds().div(3600)
step = dt_series.mode().iloc[0] if not dt_series.mode().empty else dt_series.median()
if not np.isfinite(step) or step <= 0:
    step = 0.25
dt_h = dt_series.fillna(step)

# hour_of_day
hour_of_day = pd.Series(idx.hour + idx.minute/60 + idx.second/3600, index=idx, name="hour_of_day")

# month -> season
def month_to_season(m: int) -> str:
    if m in (12, 1, 2): return "winter"
    if m in (3, 4, 5):  return "spring"
    if m in (6, 7, 8):  return "summer"
    return "autumn"
season = pd.Series([month_to_season(m) for m in idx.month], index=idx, name="season")

# t_out_c (primary: ambient_temp_C; fallback common names)
if "ambient_temp_C" in df.columns:
    t_out_c = df["ambient_temp_C"]
else:
    cand = [c for c in df.columns if c.lower() in ("t_out_c","t_out","temp_c","temperature_c")]
    t_out_c = df[cand[0]] if cand else pd.Series(np.nan, index=idx, name="t_out_c")

# price_eur_per_kwh from elec_price_EUR_per_MWh
if "elec_price_EUR_per_MWh" in df.columns:
    price_eur_per_kwh = df["elec_price_EUR_per_MWh"] / 1000.0
else:
    cand = [c for c in df.columns if ("price" in c.lower()) and ("mwh" in c.lower())]
    price_eur_per_kwh = (df[cand[0]] / 1000.0) if cand else pd.Series(np.nan, index=idx, name="price_eur_per_kwh")

# solar_gen_kw_per_kwp: irradiance proxy
if "irradiance_Wm2" in df.columns:
    solar_gen_kw_per_kwp = (df["irradiance_Wm2"].clip(lower=0) / 1000.0).fillna(0.0)
elif "P[W]" in df.columns:
    solar_gen_kw_per_kwp = (df["P[W]"].clip(lower=0) / 1000.0)
else:
    solar_gen_kw_per_kwp = pd.Series(0.0, index=idx, name="solar_gen_kw_per_kwp")

# in_work_hours (Mon–Fri, 09:00–17:00)
BUSINESS_START = 9
BUSINESS_END   = 17
in_work_hours = pd.Series(
    ((idx.dayofweek < 5) & (idx.hour >= BUSINESS_START) & (idx.hour < BUSINESS_END)).astype(int),
    index=idx, name="in_work_hours"
)

# day as INT (YYYYMMDD) — and t as sequential step
day_int = (idx.year * 10000 + idx.month * 100 + idx.day).astype(int)
t_seq = np.arange(len(df), dtype=int)

# ---------- 5) Build final dataframe with exact header order ----------
df_what_we_want = pd.DataFrame({
    "timestamp": idx,                                   # keep as column
    "t": t_seq,
    "day": day_int,                                     # INT
    "dt_h": pd.to_numeric(dt_h, errors="coerce"),
    "hour_of_day": pd.to_numeric(hour_of_day, errors="coerce"),
    "month": idx.month.astype(int),
    "season": season.astype(str),
    "dayofyear": idx.dayofyear.astype(int),
    "t_out_c": pd.to_numeric(t_out_c, errors="coerce"),
    "price_eur_per_kwh": pd.to_numeric(price_eur_per_kwh, errors="coerce"),
    "solar_gen_kw_per_kwp": pd.to_numeric(solar_gen_kw_per_kwp, errors="coerce"),
    "in_work_hours": in_work_hours.astype(int)
}, index=idx)[[
    "timestamp","t","day","dt_h","hour_of_day","month","season","dayofyear",
    "t_out_c","price_eur_per_kwh","solar_gen_kw_per_kwp","in_work_hours"
]]

# Optional: keep only column form (like your CSVs)
df_what_we_want = df_what_we_want.reset_index(drop=True)

# ---------- 6) Preview ----------
print(df_what_we_want.head())
print(df_what_we_want.dtypes)

# Optional: save
# out_path = r"C:\_projekty_repo\noc\thermal_game\data\weekXX_prices_weather_seasons_FROM_2023_RELABELED_TO_2025.csv"
# df_what_we_want.to_csv(out_path, index=False)


Wrote: C:\_projekty_repo\noc\thermal_game\data\rebuilt_from_df1_df2_conforming_to_template_2023.csv
            timestamp      t       day  dt_h  hour_of_day  month  season  \
0 2023-01-01 00:00:00  69978  20230101  0.75         0.00      1  winter   
1 2023-01-01 00:15:00  69979  20230101  0.25         0.25      1  winter   
2 2023-01-01 00:30:00  69980  20230101  0.25         0.50      1  winter   

   dayofyear  t_out_c  price_eur_per_kwh  solar_gen_kw_per_kwp  in_work_hours  
0          1   8.2825            0.07501                   0.0              0  
1          1   8.2200           -0.02995                   0.0              0  
2          1   8.2025           -0.02998                   0.0              0  
timestamp               datetime64[ns]
t                                int64
day                              int64
dt_h                           float64
hour_of_day                    float64
month                            int64
season                          object
da